In [2]:
import pandas as pd

In [3]:
region = pd.read_csv("법정동코드전체자료.txt", sep="\t")
region.법정동명

0                       서울특별시
1                   서울특별시 종로구
2               서울특별시 종로구 청운동
3               서울특별시 종로구 신교동
4               서울특별시 종로구 궁정동
                 ...         
45952    제주특별자치도 서귀포시 표선면 하천리
45953    제주특별자치도 서귀포시 표선면 성읍리
45954    제주특별자치도 서귀포시 표선면 가시리
45955    제주특별자치도 서귀포시 표선면 세화리
45956    제주특별자치도 서귀포시 표선면 토산리
Name: 법정동명, Length: 45957, dtype: object

In [4]:
region[region.폐지여부 == "존재"][region.법정동명.str.contains('서울')]

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
1107,1174010600,서울특별시 강동구 둔촌동,존재
1108,1174010700,서울특별시 강동구 암사동,존재
1109,1174010800,서울특별시 강동구 성내동,존재
1110,1174010900,서울특별시 강동구 천호동,존재


In [5]:
seoul_region = region[region.폐지여부 == "존재"][region.법정동명.str.contains('서울')].reset_index(drop=True)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [6]:
seoul_region

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
488,1174010600,서울특별시 강동구 둔촌동,존재
489,1174010700,서울특별시 강동구 암사동,존재
490,1174010800,서울특별시 강동구 성내동,존재
491,1174010900,서울특별시 강동구 천호동,존재


In [7]:
seoul_region = seoul_region[seoul_region.법정동명.str.contains('동')]

In [8]:
seoul_region = seoul_region.법정동명.str.slice(start = 6)
seoul_region = list(seoul_region)

### CRAWL

In [2]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import math
import pandas as pd

In [2]:
seoul_gu = ['강북구', '구로구','동작구', '서대문구', '종로구', '용산구', '강남구', '강동구', '금천구', '영등포구', '성북구', '양천구', '성동구', '강서구', '서초구',
            '도봉구', '은평구', '마포구', '동대문구', '중랑구', '송파구', '광진구', '관악구', '중구', '노원구']

In [5]:
# driver setting
chrom_driver = './chromedriver.exe'
# driver = webdriver.Chrome(chrom_driver) # Test

options = webdriver.chrome.options.Options()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrom_driver, options=options)

delay = 3

# crawling function
def num_restaurants_per_region(region):
    driver.implicitly_wait(2)
    url = "https://m.poing.io/search/result?keyword={}%20맛집".format(region)
    driver.get(url)
    # the number of restaurants
    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div/div/section[2]/div[1]/div/div[1]/div[2]/p/span')))
    num_restaurant = driver.find_element_by_xpath('//*[@id="app"]/div/div/div/section[2]/div[1]/div/div[1]/div[2]/p/span').text
    num_restaurant = int(num_restaurant[1:-1])

    print("# of Restaurants in {}: {}".format(region, num_restaurant))
    driver.quit

    return num_restaurant


def crawl_save(num_restaurant, region):
    result_df = pd.DataFrame(columns=["Region", "Rating", "Category", "Review"])
    for i in range(num_restaurant):

        if (i+1) % 100 == 0 and i != 0:
            result_df.to_parquet("{}th_restaurant_in_{}.parquet".format(i+1, region))
            print("Checkpoint!")

        try:
            url = "https://m.poing.io/search/result?keyword={}%20맛집".format(region)
            driver.get(url)
            cur_num_restaurant = 0
            
            # scrawl web page
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div/div/section[2]/div[2]/div/div')))
            restaurant_list = driver.find_element_by_xpath('//*[@id="app"]/div/div/div/section[2]/div[2]/div/div')
            
            if i >= 20:
                while num_restaurant != cur_num_restaurant and ((i // 20) + 1) * 20 != cur_num_restaurant: # too slow
                    driver.execute_script('arguments[0].scrollBy(0, document.body.scrollHeight);', restaurant_list)

                    WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'place-list-container')))
                    restaurants = driver.find_elements_by_class_name("place-list-container")
                    
                    cur_num_restaurant = len(restaurants)
                    # print(cur_num_restaurant)
                    
                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'stitle')))
                categories = [stitle.text.split()[3] if stitle.text.split()[2] == "·" else  stitle.text.split()[2] for stitle in driver.find_elements_by_class_name("stitle") ]

            # restaurant
            else:
                WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'place-list-container')))
                restaurants = driver.find_elements_by_class_name("place-list-container")

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'stitle')))
                categories = [stitle.text.split()[3] if stitle.text.split()[2] == "·" else  stitle.text.split()[2] for stitle in driver.find_elements_by_class_name("stitle") ]
                
                cur_num_restaurant = len(restaurants)
                # print(cur_num_restaurant)

            restaurants[i].click()
            time.sleep(2)

            # check the number of reviews
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'count')))
            review_count = int(driver.find_element_by_class_name('count').text[1:-1])
            # print("review count: {}".format(review_count))

            if review_count == 0:
                print("{}th Restaurant => No Review!".format(i + 1))
                continue

            # enter review page
            try:
                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div/div/section[4]/h2/button')))
                driver.find_element_by_xpath('//*[@id="app"]/div/div/div/section[4]/h2/button').send_keys(Keys.ENTER)
            except:
                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div/div/section[7]/h2/button')))
                driver.find_element_by_xpath('//*[@id="app"]/div/div/div/section[7]/h2/button').send_keys(Keys.ENTER)
            
            # waiting for loading
            driver.implicitly_wait(delay)
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'place-review-list-wrap')))
            review_list = driver.find_element_by_class_name("place-review-list-wrap")
            cur_num_reviews = 0
            while cur_num_reviews != review_count:
                # print("cur review cnt: {}, review count: {}".format(cur_num_reviews, review_count))
                driver.execute_script('arguments[0].scrollBy(0, document.body.scrollHeight);', review_list)
                
                WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'review-list-contents__text')))
                reviews = driver.find_elements_by_class_name("review-list-contents__text")
                
                cur_num_reviews = len(reviews)
                time.sleep(1)
        
            # crawling reviews  
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'review-list-contents__text')))
            reviews = [review.text for review in driver.find_elements_by_class_name('review-list-contents__text')]
            # print(reviews)

            # crawling ratings
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'star-count')))
            ratings = [round(float(review.text)) for review in driver.find_elements_by_class_name('star-count')]
            # print(ratings)

            # crawling category
            # print(categories[i])

            # save dataframe
            data = {"Region":[region for _ in reviews], "Rating":ratings, "Category": [categories[i] for _ in reviews], "Review": reviews}
            result_df = result_df.append(pd.DataFrame(data))
            
        except:
            print("{}th Restaurant => Error occur!".format(i + 1))
            continue
        print("Searching {}th restaurant in {}: {}% complete ({}reviews crawling)".format(i+1, region, (i + 1)*100 / num_restaurant , len(result_df)))
    return result_df

num_reviews = 0

df = pd.DataFrame(columns=["Region", "Rating", "Category", "Review"])
num_restaurant = 0
region = "남양주"

while num_restaurant == 0:
    num_restaurant = num_restaurants_per_region(region)  
    if num_restaurant == 0:
        print("Retry")  
    else:
        print("Start Crawling")

df = df.append(crawl_save(num_restaurant, region))
num_reviews += len(df)
df.to_parquet("{}.parquet".format(region))
print("{} Crawling finish => {} reviews".format(region, num_reviews))

# of Restaurants in 남양주: 30
Start Crawling
1th Restaurant => No Review!
Searching 2th restaurant in 남양주: 6.666666666666667% complete (1reviews crawling)
3th Restaurant => Error occur!
Searching 4th restaurant in 남양주: 13.333333333333334% complete (4reviews crawling)
Searching 5th restaurant in 남양주: 16.666666666666668% complete (7reviews crawling)
6th Restaurant => No Review!
7th Restaurant => No Review!
Searching 8th restaurant in 남양주: 26.666666666666668% complete (13reviews crawling)
Searching 9th restaurant in 남양주: 30.0% complete (20reviews crawling)
Searching 10th restaurant in 남양주: 33.333333333333336% complete (26reviews crawling)
Searching 11th restaurant in 남양주: 36.666666666666664% complete (30reviews crawling)
Searching 12th restaurant in 남양주: 40.0% complete (34reviews crawling)
Searching 13th restaurant in 남양주: 43.333333333333336% complete (38reviews crawling)
Searching 14th restaurant in 남양주: 46.666666666666664% complete (40reviews crawling)
Searching 15th restaurant in 남양주: 50